In [27]:
using PyPlot, HDF5;
using Statistics
include("sacadatos.jl")

devuelvedatos (generic function with 1 method)

In [6]:
nomine = "280518_19.hf5";

In [7]:
datos = h5open(nomine, "r");

In [8]:
names(datos)

5-element Array{String,1}:
 "Im_primaryIm_sec"
 "Im_sec"          
 "channels"        
 "comment"         
 "description"     

In [13]:
aux=read(datos["Im_primaryIm_sec"]);
aux["description"]

1-element Array{HDF5.HDF5Compound{1},1}:
 HDF5.HDF5Compound{1}((17,), ("n_sections",), (Int32,))

In [24]:
sweps = devuelvedatos(datos, "Im_primaryIm_sec");

la descripcion de tus barridas de datos son las siguientes
HDF5.HDF5Compound{3}[HDF5Compound{3}((0.04, "ms", "mV"), ("dt", "xunits", "yunits"), (Float64, String, String))]


In [25]:
function encuentraespigas(datos::Dict, umbral=-10)
   result = Dict{String, Any}()
    for (key,value) in datos
        aux=findall(x->x>umbral, value)
        if length(aux)>0
            result[key]=aux
        end
    end
    return result
end

encuentraespigas (generic function with 2 methods)

In [26]:
pickIn = encuentraespigas(sweps,-5)

Dict{String,Any} with 12 entries:
  "section_06" => [2969, 2970, 2971, 2972, 2973, 2974, 2975, 2976, 2977, 2978  …
  "section_13" => [2596, 2597, 2598, 2599, 2600, 2601, 2602, 2603, 2604, 2605  …
  "section_10" => [2625, 2626, 2627, 2628, 2629, 2630, 2631, 2632, 2633, 2634  …
  "section_14" => [2590, 2591, 2592, 2593, 2594, 2595, 2596, 2597, 2598, 2599  …
  "section_12" => [2630, 2631, 2632, 2633, 2634, 2635, 2636, 2637, 2638, 2639  …
  "section_05" => [3216, 3217, 3218, 3219, 3220, 3221, 3222, 3223, 3224, 3225  …
  "section_16" => [2565, 2566, 2567, 2568, 2569, 2570, 2571, 2572, 2573, 2574  …
  "section_08" => [2774, 2775, 2776, 2777, 2778, 2779, 2780, 2781, 2782, 2783  …
  "section_09" => [2724, 2725, 2726, 2727, 2728, 2729, 2730, 2731, 2732, 2733  …
  "section_15" => [2568, 2569, 2570, 2571, 2572, 2573, 2574, 2575, 2576, 2577  …
  "section_07" => [2945, 2946, 2947, 2948, 2949, 2950, 2951, 2952, 2953, 2954  …
  "section_11" => [2680, 2681, 2682, 2683, 2684, 2685, 2686, 2687, 2688, 26

In [ ]:
function rasterplot(lespigas)
    # hace un rasterplot como diccionario a partir de una lista de espigas.
raster = Dict{Any, Any}() #preallocation
m = 0
while m <= length(s)
    # generar nombre de sweep
    if m < 10
        aux_0 = "section_0"
    else
        aux_0 = "section_"
    end
    aux0 = (string(aux_0,string(m)))
    m += 1
    #############################################################################################
    # what sweeps really have voltage values above the threshold
    aux_1 = get(pickIn, aux0, 0)
    if aux_1 == 0
        continue
    end
    #############################################################################################
    aux1 = parse(Int,replace(aux0, "section_" => ""))
    # obtener el voltaje de ese sweep
     ejemplo = s[aux0]; 
    # obtener los valores que sobrepasan el umbral de ese sweep
    selection = pickIn[aux0];
    # obtener los valores maximos de voltaje de ese sweep
    i = 1
    inicio = i
    j = 1
    for_raster = Dict{Any, Any}() #preallocation
    while i < length(selection)
        if selection[i + 1] == selection[i] + 1
            i += 1
        else
            fin = i
            trazo = selection[inicio:fin]
            voltaje = ejemplo[trazo]
            aux = findmax(voltaje)
            for_raster[j] = trazo[aux[2]]
            j += 1
            inicio = fin + 1
            i += 1
        end
    end
    trazo = selection[inicio:length(selection)]
    voltaje = ejemplo[trazo]
    aux = findmax(voltaje)
    for_raster[j] = trazo[aux[2]];
    ############
    entry0 = for_raster[1]
    for i = 2:length(for_raster)
        entry0 = [entry0 for_raster[i]]
    end
    ############
    raster[aux1] = [aux0 entry0] # se almacenan los valores maximos de voltaje registrados por cada sweep
end
    return raster 
end

In [ ]:
raster=rasterplot(pickIn)

In [ ]:
import PlotlyJS
p=PlotlyJS.plot(s["section_16"], marker_color="blue", line_width=1)
show(p)

In [ ]:
plot(ejemplo[2792:3020])